# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902036


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.42s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:48,  1.80s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:32,  1.28s/it]

Rendering models:  20%|██        | 6/30 [00:05<00:22,  1.07it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  7.64it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  7.06it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.63it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  8.14it/s]

kayleebug2017                         0.000262
Mouring                               0.000345
Lukes0404                             0.003308
Linda_J._Berkel                       0.000326
Thedunkmasta                          0.001949
not-logged-in-0997cfcba8da23453381    0.000672
HexBerry                              0.000112
ElisabethB                            0.246441
not-logged-in-ab1612bc18cd7aab85db    0.000114
not-logged-in-1bfb5ea67e2d65ad84ff    0.001044
not-logged-in-5a8bb7be433678ade709    0.001209
Jiayao                                0.000539
mariafarhat                           0.001368
not-logged-in-73b973978dc7c9e2e5ac    0.000299
gabrielweiss                          0.000513
aidenr                                0.000606
rhondacard                            0.001148
Lavadude                              0.001650
not-logged-in-3429675089c67c4fccb8    0.291467
MarikaKielbiowska                     0.016378
vceccare                              0.004590
FR4N1O       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())